<a href="https://colab.research.google.com/github/mukundan1989/newsmod1.2/blob/main/NewsModel_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import http.client
import json
import time

# Initialize connection
conn = http.client.HTTPSConnection("seeking-alpha.p.rapidapi.com")

# Set headers
headers = {
    'x-rapidapi-key': "42d8e62a6bmsh25ee08e3260a166p1ce1acjsn6f3b6d6f7d55",
    'x-rapidapi-host': "seeking-alpha.p.rapidapi.com"
}

# Parameters
size = 20  # Number of articles per request
since_timestamp = 1722470400  # Set your starting timestamp
until_timestamp = 1724976000  # Set your ending timestamp
all_news_data = []

# Loop to paginate through results
while True:
    # Request data with current 'since' and 'until' timestamps
    conn.request("GET", f"/news/v2/list-by-symbol?until={until_timestamp}&since={since_timestamp}&size={size}&id=NVDA", headers=headers)
    res = conn.getresponse()

    # Read the response and try to decode it as JSON
    try:
        data = json.loads(res.read().decode("utf-8"))
    except json.JSONDecodeError as e:
        # Handle JSON decode error by printing the raw response and error message
        print("Failed to decode JSON:", e)
        print("Raw response:", res.read().decode("utf-8"))
        break  # Exit the loop if there's a JSON error

    # Check if there are articles in the response
    if not data['data']:
        break  # Exit if no more articles are available

    # Add the fetched data to the list
    all_news_data.extend(data['data'])

    # Update `until_timestamp` for the next batch (adjust by day or specific interval)
    until_timestamp -= 86400  # Move one day back (86400 seconds in a day)

    # Optional: Sleep to avoid rate limits
    time.sleep(2)

# Optional: Save or process `all_news_data`
print(f"Total articles retrieved: {len(all_news_data)}")

# Example: Write to CSV
import csv
with open('nvda_news_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['ID', 'Publish Date', 'Title', 'Author ID', 'Comment Count', 'Primary Tickers', 'Secondary Tickers', 'Image URL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for item in all_news_data:
        writer.writerow({
            'ID': item['id'],
            'Publish Date': item['attributes']['publishOn'],
            'Title': item['attributes']['title'],
            'Author ID': item['relationships']['author']['data']['id'],
            'Comment Count': item['attributes']['commentCount'],
            'Primary Tickers': ', '.join([t['type'] for t in item['relationships']['primaryTickers']['data']]),
            'Secondary Tickers': ', '.join([t['type'] for t in item['relationships']['secondaryTickers']['data']]),
            'Image URL': item['attributes'].get('gettyImageUrl', 'N/A')
        })

print("CSV file has been created successfully.")


In [ ]:
import http.client
import csv
import pandas as pd
import time

# Define file path
csv_file_path = "/content/nvda_news_data.csv"

# Load the CSV file
df = pd.read_csv(csv_file_path)

# Ensure there is an 'ID' column
if 'ID' not in df.columns:
    raise ValueError("The CSV file does not contain an 'ID' column.")

# Initialize Content column
if 'Content' not in df.columns:
    df['Content'] = None

# API details
api_key = "eece906463msh0e8644a8935d437p1bbbb6jsn96dc85f57450"
api_host = "seeking-alpha.p.rapidapi.com"

# Initialize HTTP connection
conn = http.client.HTTPSConnection(api_host)

# Loop through each ID and fetch content
for index, row in df.iterrows():
    news_id = row['ID']

    # Skip if content is already fetched
    if pd.notna(row['Content']):
        continue

    try:
        # Make API request
        headers = {
            'x-rapidapi-key': api_key,
            'x-rapidapi-host': api_host
        }
        conn.request("GET", f"/news/get-details?id={news_id}", headers=headers)
        res = conn.getresponse()
        data = res.read()

        # Decode and store the content
        df.at[index, 'Content'] = data.decode('utf-8')
        print(f"Fetched content for ID: {news_id}")

        # Pause to avoid rate limits
        time.sleep(1)  # Adjust delay based on API limits if necessary

    except Exception as e:
        print(f"Error fetching content for ID {news_id}: {e}")

# Save updated CSV
df.to_csv(csv_file_path, index=False)
print("Content fetched and saved successfully!")


In [ ]:
import pandas as pd
import json
import re

# Function to clean HTML tags
def clean_html(raw_html):
    clean_regex = re.compile('<.*?>')  # Regex to match HTML tags
    return re.sub(clean_regex, '', raw_html)

# Function to extract article content from the JSON
def extract_content(full_data):
    try:
        # Load the JSON content
        data = json.loads(full_data)
        content = data.get("data", {}).get("attributes", {}).get("content", "")

        # Clean HTML from content
        cleaned_content = clean_html(content)

        # Define potential ending markers
        ending_markers = ["More on", "Read more", "See also", "Learn more", "Related articles"]

        # Find the first occurrence of any ending marker and truncate content
        for marker in ending_markers:
            if marker in cleaned_content:
                cleaned_content = cleaned_content.split(marker)[0]
                break  # Stop after finding the first marker

        return cleaned_content.strip()  # Remove any leading/trailing spaces
    except (json.JSONDecodeError, TypeError):
        return None  # Return None if parsing fails

# Load your data (assumes CSV file for example purposes)
input_file = "/content/nvda_news_data.csv"  # Replace with your file path
df = pd.read_csv(input_file)

# Apply the extraction function to the "FullData" column
df["Extracted"] = df["FullData"].apply(extract_content)

# Save the updated table to a new file
output_file = "updated_table_with_extracted.csv"
df.to_csv(output_file, index=False)

print(f"Processed table saved to {output_file}")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import re

# Load the original PhraseBank
phrase_bank_path = '/content/FinancialPhraseBank.csv'
column_names = ['Strong Positive', 'Positive', 'Neutral', 'Negative', 'Strong Negative']
phrase_bank_df = pd.read_csv(phrase_bank_path, sep=',', names=column_names)

# Reshape original PhraseBank
phrase_bank = []
sentiment_mapping = {
    'Strong Positive': 'Strong Positive',
    'Positive': 'Positive',
    'Neutral': 'Neutral',
    'Negative': 'Negative',
    'Strong Negative': 'Strong Negative'
}

for sentiment, phrases in phrase_bank_df.items():
    for phrase in phrases.dropna():
        phrase_bank.append({'Phrase': phrase, 'Sentiment': sentiment_mapping[sentiment]})

phrase_bank_df = pd.DataFrame(phrase_bank)

# Load the new PhraseBank2
phrase_bank2_path = '/content/FinancialPhraseBank2.csv'
column_names2 = ['Positive', 'Negative', 'Neutral']
phrase_bank2_df = pd.read_csv(phrase_bank2_path, sep=',', names=column_names2)

# Reshape PhraseBank2
phrase_bank2 = []
for sentiment in column_names2:
    for phrase in phrase_bank2_df[sentiment].dropna():
        phrase_bank2.append({'Phrase': phrase, 'Sentiment': sentiment})

phrase_bank2_df = pd.DataFrame(phrase_bank2)

# Set up FinBERT
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Helper function to truncate content
def truncate_content(content, tokenizer, max_length=512):
    inputs = tokenizer(content, max_length=max_length, truncation=True, return_tensors="pt")
    return tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)

# Analyze sentiment with FinBERT
def analyze_sentiment(content):
    truncated_content = truncate_content(content, tokenizer)
    result = sentiment_pipeline(truncated_content)[0]
    return result['label'], result['score']

# Calculate sentiment matches for a given PhraseBank
def get_phrase_sentiment(content, phrase_bank_df):
    matched_sentiments = []
    for _, row in phrase_bank_df.iterrows():
        if re.search(r'\b' + re.escape(row['Phrase']) + r'\b', content, re.IGNORECASE):
            matched_sentiments.append(row['Sentiment'])
    return matched_sentiments if matched_sentiments else ["None"]

# Process a single row
def process_row(row, phrase_bank_df, phrase_bank2_df):
    content = row['Extracted']
    if not isinstance(content, str) or not content.strip():
        return None, None, None, None  # Handle empty or invalid content

    # FinBERT analysis
    finbert_label, finbert_score = analyze_sentiment(content)

    # PhraseBank1 results
    phrasebank1_sentiments = "; ".join(get_phrase_sentiment(content, phrase_bank_df))

    # PhraseBank2 results
    phrasebank2_sentiments = "; ".join(get_phrase_sentiment(content, phrase_bank2_df))

    return finbert_label, finbert_score, phrasebank1_sentiments, phrasebank2_sentiments

# Load the input file
input_file = '/content/updated_table_with_extracted.csv'
output_file = '/content/updated_table_with_scores.csv'

data = pd.read_csv(input_file)

# Apply the processing function to each row
results = data.apply(
    lambda row: process_row(row, phrase_bank_df, phrase_bank2_df), axis=1
)

# Add columns for FinBERT, PhraseBank1, and PhraseBank2 results
data['FinBERT Label'], data['FinBERT Score'], data['PhraseBank1 Results'], data['PhraseBank2 Results'] = zip(*results)

# Save the updated DataFrame to a new CSV file
data.to_csv(output_file, index=False)

print(f"Analysis completed. Updated file saved at {output_file}")


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Analysis completed. Updated file saved at /content/updated_table_with_scores.csv
